This notebook:

- reads graphs from the PyTorch Geometric dataset, into Networkx versions of the graphs, then computes the node betweenness centrality (and edge betweenness centrality).
- then transforms the Networkx graph into a json graph object
- it also creates id_map, which assign every node identifier to a consecutive integer
- it also creates class_maps, which will save the betweeness centrality of each node
- finally an edge version, converts graph into  node representation of vertices  and saves the edge betweenees in the class_maps

## 1. Code base

In [3]:
import importlib
import torch
from torch_geometric.data import DataLoader
import networkx as nx
from torch_geometric.data import Data
import time
import pickle
from torch_geometric.data import InMemoryDataset
import numpy as np
import traceback

      
    
def writeAdjacencyMatrixToDisk(G, filename='temp_adjacency_matrix.txt'):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    f = open(filename,'w')
    _ni=-1
    newline = False
    theline = []
    careturn = ""
    for ei in range(G.edge_index.size()[1]):
        if int(G.edge_index[0,ei].item()) != _ni:
            newline=True
            _ni=int(G.edge_index[0,ei].item())
            
        else:
            newline=False
            
            
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if newline:
            f.write(''.join(theline))
            #print(''.join(theline))
            #print(" --> "+str(_ni))
            theline =[]
            theline.append(careturn+ni+" ")
            theline.append(vi+" ")
            careturn = "\n"
        else:
            theline.append(vi+" ")
        # print("({},{})".format(ni,vi))
    
    
def nx_createNxGraphInMem(G):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    g = nx.MultiGraph()
   
    for ei in range(G.edge_index.size()[1]):    
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        g.add_edge(ni,vi)
    return g
    
def nx_verifyEdges(G, g):
    for ei in range(G.edge_index.size()[1]):
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if (ni,vi,0) not in list(g.edges):
            if (vi,ni,1) not in list(g.edges):
                print("Error {} not in networkx graph".format((ni,vi)))
            
        

def nx_compute_edge_betweenness(G):
    
    #print(list(G.edges)[:10])
    G_components = nx.connected_component_subgraphs(G)
    G_mc = list(G_components)[0]  
    eb_dict_res = {}
    eb_dict = nx.edge_betweenness_centrality(G_mc)
    
    # if there are more connected components...
    if len(list(G_components))>1:
        print("WARNING connected components: ",len(list(G_components)))
    
    eb_dict_res.update(eb_dict)
    
        
    return eb_dict_res

def nx_compute_node_betweenness(G):
    
    #print(list(G.edges)[:10])
    G_components = nx.connected_component_subgraphs(G)
    G_mc = list(G_components)[0]  
    eb_dict_res = {}
    eb_dict = nx.betweenness_centrality(G_mc)
    
    # if there are more connected components...
    if len(list(G_components))>1:
        print("WARNING connected components: ",len(list(G_components)))
    
    eb_dict_res.update(eb_dict)
    
        
    return eb_dict_res


def update_edge_betweenness(G, eb_dict):
    """
        FOR UNDIRECTED GRAPHS
    
        G.edge_attr must contain the edge betweenness values 
        for each edge
        
        G.y must contain it also.. (it is a copy of the edge betweenness..)
        this could help the training phase
        
        Size restrictions:
        - Given the size of the graphs, is it better to just transform the 
        object instead to write a new one?
        - also just use G.y? but for GNN algorithms..not sure
        
        new_edg_attr will be size [num edges, 1]
        and must be sorted in accordance to G.edge_index
        
    
    """
    
    new_edg_attr = []
    for i in range(len(G.edge_index[0])):
        ni = G.edge_index[0][i]
        vi = G.edge_index[1][i]
        
        if ni and vi:
            ni=str(ni.item())
            vi=str(vi.item())
            #print((ni,vi))
            try:
                new_edg_attr.append([eb_dict[(ni,vi)]])
            except:
                try:
                    new_edg_attr.append([eb_dict[(vi,ni)]])
                except:
                    #print("ERROR {} and {} not found!".format((ni,vi),(vi,ni)))
                    new_edg_attr.append([0])
        else:
            new_edg_attr.append([0])

    new_edg_attr = torch.FloatTensor(new_edg_attr)
    
    #newG = Data(
    #    x=G.x, 
    #    edge_index=G.edge_index, 
    #    edge_attr=new_edg_attr,
    #    y=new_edg_attr)
    
    #G.edge_attr = new_edg_attr
    G.y = new_edg_attr
    
    return G


def update_node_betweenness(G, eb_dict):
    """
        Get nodes keys from eb_dict and get their betweenness centrality
        G.y will have all centralities of al lnodes following the order
        of a list of the nodes sorted by id

        add spaces in between!

    """
    betweennesses = []
    nodes = sorted([int(k) for k in eb_dict.keys()])
    for node in range(nodes[-1]+1):
        try:
            betweennesses.append(eb_dict[str(node+1)])    
        except:
            betweennesses.append(0.0)
            
    G.y = torch.FloatTensor(betweennesses)
    return G

def get_betweenness_into_dict(G):
    """
        FOR UNDIRECTED GRAPHS
    """
    
    eb_dict ={}
    for i in range(len(G.edge_index[0])):
        ni = G.edge_index[0][i]
        vi = G.edge_index[1][i]
        
        if ni and vi:
            ni=str(ni.item())
            vi=str(vi.item())
            eb_dict[(ni,vi)] = float(G.y[i].item())
    return eb_dict


def pyTorchGeometricDatasetToNx(G,suffix=0):
    """
        Alternatives:
            - to disk, to nx, then dict of betweenness
            - transform in memory
            - directly pickle a G object with the betweenness
    """
    prefix = 'temp_aj_m'
    # 1. PyTorch Geometric graph -> nx -> compute betweenness 
    #             -> PyTorch Geom with target the betweenness-------
    # Transform to networkx graph
    # write to adjacency matrix on disk
    writeAdjacencyMatrixToDisk(G, filename=prefix+str(suffix)+'.txt')

    # load into a networkx graph object
    g2 = nx.read_adjlist(prefix+str(suffix)+'.txt')
    #g2 = nx_createNxGraphInMem(G)
    
    return g2

def computeBetweenness(G,suffix=0):
    """
        Alternatives:
            - to disk, to nx, then dict of betweenness
            - transform in memory
            - directly pickle a G object with the betweenness
    """
    prefix = 'temp_aj_m'
    # 1. PyTorch Geometric graph -> nx -> compute betweenness 
    #             -> PyTorch Geom with target the betweenness-------
    # Transform to networkx graph
    # write to adjacency matrix on disk
    writeAdjacencyMatrixToDisk(G, filename=prefix+str(suffix)+'.txt')

    # load into a networkx graph object
    g2 = nx.read_adjlist(prefix+str(suffix)+'.txt')
    #g2 = nx_createNxGraphInMem(G)

    # compute node betweenness centrality
    eb_dict = nx_compute_node_betweenness(g2)
    #print("eb_dict",eb_dict)
    
    # write node betweenness back to PyTorch Geometric graph
    update_node_betweenness(G,eb_dict)
    #return G
    


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #data_list = [G.x, G.edge_index, G.test_mask, G.train_mask, G.val_mask, G.y, G.batch]
        f.close()
        #print("root ", root, " name ", name)
        #print("setting a self.name in the object!")
        #self.name = name
        #print(dir(self))
        #super(MyOwnDataset, self).__init__(root,transform, pre_transform)
        #self.data = torch.load(self.processed_paths[0])
        
        
    
    
    @property
    def raw_file_names(self):
        return ['PPI0.pickle']

    @property
    def processed_file_names(self):
        return ['PPI0']

    def download(self):
        # Download to `self.raw_dir`.
        return True

    def process(self):
        # Read data into huge `Data` list.
        
        # unpickle the graph
        print("going to unpickle")
        f = open(self.name, 'rb')
        G = pickle.load(f) 
        #data_list = [G.x, G.edge_index, G.test_mask, G.train_mask, G.val_mask, G.y, G.batch]
        f.close()
        
        #if self.pre_filter is not None:
        #    data_list [data for data in data_list if self.pre_filter(data)]

        #if self.pre_transform is not None:
        #    data_list = [self.pre_transform(data) for data in data_list]

        #data, slices = self.collate(data_list)
        torch.save(G, self.processed_paths[0])
        
        
class MyOwnDataset2():
    def __init__(self,  name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        f.close()
    
def loadDataset(collection, name=None, split=None):
    # import datasets
    themodule = importlib.import_module("torch_geometric.datasets")
    # get the function corresponding to collection
    method_to_call = getattr(themodule, collection)
    try:
        if name:
            return method_to_call(root='./data/'+str(collection), name=name)
        elif split:
            return method_to_call(root='./data/'+str(collection), split=split)
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except:
        traceback.print_tb()
        if name:
            return method_to_call( name=name)
        elif split:
            return method_to_call(split=split)
        else:
            return method_to_call()         
        
    
def createDataset(x, edge_index):
    return Data(x=x, edge_index=edge_index)
    

def createDatasetFromNX(g, xlen, undirected=True):
    # get edge list
    edges = g.edges
    edge_list_1 = []
    edge_list_2 = []
    for e in edges:
        # node id must be an int
        edge_list_1.append(int(e[0])) 
        edge_list_2.append(int(e[1]))
        if undirected:
            edge_list_1.append(int(e[1])) 
            edge_list_2.append(int(e[0]))
            
        
    edge_index = torch.tensor([ edge_list_1,
                                edge_list_2], dtype=torch.long)
    
    # create single 1 feature for each node
    n = xlen
    x = [[1.0] for i in range(n)]
    x = torch.tensor(x, dtype=torch.float)
    
    return createDataset(x, edge_index)
                         
def createDatasetFromNXwithTarget(g,y,xlen, undirected=True):
    dataset =  createDatasetFromNX(g,xlen, undirected)
    y = torch.FloatTensor(y)
    dataset.y = y 
    return dataset
                   

## 2. GraphSAGE TF Graph datasets

### Tests

### Implementation

In [25]:
import json
from networkx.readwrite import json_graph
import random


def  edgeToNodeTransform(g,verif=False):
    """
    # transform:
        #  1-every edge is a node
        #  2.every edge that share a node creates a link between them
        #º 3- id_map with an ide for each "edge_node"
        #  3- class_map with edge-betwweennes and id
        
        
    1) create a new graph g2
    2) iter edge of g 
    2.1) and add as node in g2
    2.2) iter edges of g looking for node in original edge, 
         add them as nodes in g2 and add links in g2
        
    """
    g2 = nx.Graph()
    i=0
    for e in g.edges_iter():
        g2.add_node(i)
        g2.node[i]['old_edge']=e
        i+=1
        
        
    for n in g2.nodes():
        # now for each node in the old_edge
        # find other edges that are neighbors
        # and add the corresponding new edge in g2

        e = g2.node[n]['old_edge']
        for n2 in g2.nodes():
            if n==n2:
                continue
            e2 = g2.node[n2]['old_edge']
            if e2[0] == e[0] or e2[1] == e[0] \
               or e2[0] == e[1] or e2[1] == e[1]:
                g2.add_edge(n,n2)
                # this will add 2 times n,n2 and n2,n edges...
                # should we remove it?
        
    return g2

def createGraphSAGEDataset(g,dname,betweenness, undirected=True):
    
    # compute its node and edge betweenness
    y =[]
    x=[]
    xlen=len(g.nodes())
    
    
    class_map = {}
    id_map ={}
    i = 0
    
    if betweenness == 'node':
        nx_betweenness = nx.betweenness_centrality(g)
        #y = [ v for k,v in nx_betweenness.items()]
        y = []
        for k,v in nx_betweenness.items():
            y.append(v)
            id_map[str(k)] = i
            class_map[str(k)] = v
            i+=1
    elif betweenness == 'classes':
        nx_betweenness = nx.betweenness_centrality(g)
        
        # discretize in 10 classes and assign 
        
        y = []
        for k,v in nx_betweenness.items():
            y.append(v)
            theclass = [0]*10
            #print(theclass)
            if v*1000 > 9.0:
                theclass[9]=1
            elif v*1000 > 8.0:
                theclass[8]=1
            elif v*1000 > 7.0:
                theclass[7]=1
            elif v*1000 > 6.0:
                theclass[6]=1
            elif v*1000 > 5.0:
                theclass[5]=1
            elif v*1000 > 4.0:
                theclass[4]=1
            elif v*1000 > 3.0:
                theclass[3]=1
            elif v*1000 > 2.0:
                theclass[2]=1
            elif v*1000 > 1.0:
                theclass[1]=1
            id_map[str(k)] = i
            class_map[str(k)] = theclass
            i+=1
        
    else:
        nx_edge_betweenness = nx.edge_betweenness_centrality(g)
        
        # this part needs a rethinking, edge_betweenness goest to edges
        #class_map[str(k)] = v
        # we would need to TRANSFORM the graph and then save the betweenness 
        # as a node betweenness
        g2 = edgeToNodeTransform(g)
        g = g2
        
        y = []
        for n in g2.nodes():
            id_map[str(n)] = n
            class_map[str(n)] = nx_edge_betweenness[g2.node[n]['old_edge']]
            y.append(class_map[str(n)])
            
        xlen=len(y)
        
    
    
    # add 'val' and 'test' attributes to all nodes
    for nd in g.nodes():
        # random uniform with 3 values
        node = g.node[nd]
        node['val']=0
        node['test']=0
        node['train']=0
        rand_elem = random.choice(['val','test','train'])
        node[rand_elem]=1
    
    # translate into a Networkx graph json object
    jg = json_graph.node_link_data(g)
    
    # and write to disk as a json object
    with open(dname+"-G.json", "w+") as write_file:
        json.dump(jg, write_file)
    
    # id_map
    with open(dname+"-id_map.json", "w+") as write_file:
        json.dump(id_map, write_file)
    
    # class_map
    with open(dname+"-class_map.json", "w+") as write_file:
        json.dump(class_map, write_file)
    
    

In [34]:
# test graph transform
print("\ntest1")
G = nx.Graph()
G.add_path([0,1,2,3])
print(list([e for e in G.edges_iter()]))
G = edgeToNodeTransform(G)
print(list([e for e in G.edges_iter()]))

print("\ntest2")
G = nx.Graph()
G.add_path([0,1,2,3])

G.add_path([4,5,6])
G.add_edge(0,4)
G.add_edge(3,6)
print(list([e for e in G.edges_iter()]))
G = edgeToNodeTransform(G)
print(list([e for e in G.edges_iter()]))


print("\ntest3")
G = nx.Graph()
G.add_path([0,1,3])

G.add_path([4,5,6])
G.add_edge(0,4)
G.add_edge(3,6)

G.add_path([7,8,9])
G.add_edge(0,7)
G.add_edge(3,9)

print(list([e for e in G.edges_iter()]))
G = edgeToNodeTransform(G)
print("nodes",G.nodes(data=True))
print(list([e for e in G.edges_iter()]))


test1
[(0, 1), (1, 2), (2, 3)]
[(0, 1), (1, 2)]

test2
[(0, 1), (0, 4), (1, 2), (2, 3), (3, 6), (4, 5), (5, 6)]
[(0, 1), (0, 2), (1, 5), (2, 3), (3, 4), (4, 6), (5, 6)]

test3
[(0, 1), (0, 4), (0, 7), (1, 3), (3, 6), (3, 9), (4, 5), (5, 6), (7, 8), (8, 9)]
nodes [(0, {'old_edge': (0, 1)}), (1, {'old_edge': (0, 4)}), (2, {'old_edge': (0, 7)}), (3, {'old_edge': (1, 3)}), (4, {'old_edge': (3, 6)}), (5, {'old_edge': (3, 9)}), (6, {'old_edge': (4, 5)}), (7, {'old_edge': (5, 6)}), (8, {'old_edge': (7, 8)}), (9, {'old_edge': (8, 9)})]
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 6), (2, 8), (3, 4), (3, 5), (4, 5), (4, 7), (5, 9), (6, 7), (8, 9)]


In [24]:
er = nx.erdos_renyi_graph(100, 0.15)
ws = nx.watts_strogatz_graph(30, 3, 0.1)
ba = nx.barabasi_albert_graph(100, 5)
red = nx.random_lobster(100, 0.9, 0.9)

createGraphSAGEDataset(er,'./precomputed-graphSAGE-TF/er_100_0_15_eb','edge', undirected=False)



In [21]:
er = nx.erdos_renyi_graph(100, 0.15)
createGraphSAGEDataset(er,'./precomputed-graphSAGE-TF/er_100_0_15_nb_discrete','classes', undirected=False)


In [4]:
er = nx.erdos_renyi_graph(100, 0.15)
createGraphSAGEDataset(er,'er_100_0_15_nb','node')
er = nx.erdos_renyi_graph(100, 0.45)
createGraphSAGEDataset(er,'er_100_0_45_nb','node')
er = nx.erdos_renyi_graph(1000, 0.15)
createGraphSAGEDataset(er,'er_1000_0_15_nb','node')
er = nx.erdos_renyi_graph(1000, 0.45)
createGraphSAGEDataset(er,'er_1000_0_45_nb','node')


ws = nx.watts_strogatz_graph(30, 3, 0.1)
createGraphSAGEDataset(ws,'ws_30_3_0_1_nb','node')
ws = nx.watts_strogatz_graph(100, 3, 0.1)
createGraphSAGEDataset(ws,'ws_100_3_0_1_nb','node')
ws = nx.watts_strogatz_graph(1000, 3, 0.1)
createGraphSAGEDataset(ws,'ws_1000_3_0_1_nb','node')


ws = nx.watts_strogatz_graph(1000, 10, 0.1)
createGraphSAGEDataset(ws,'ws_1000_10_0_1_nb','node')


ba = nx.barabasi_albert_graph(100, 5)
createGraphSAGEDataset(ba,'ba_100_5_nb','node')
ba = nx.barabasi_albert_graph(1000, 5)
createGraphSAGEDataset(ba,'ba_1000_5_nb','node')



#er = nx.erdos_renyi_graph(4000, 0.15)
#createGraphSAGEDataset(er,'er_4000_0_15_nb','node')
#er = nx.erdos_renyi_graph(4000, 0.35)
#createGraphSAGEDataset(er,'er_4000_0_35_nb','node')
#ws = nx.watts_strogatz_graph(4000, 3, 0.1)
#createGraphSAGEDataset(ws,'ws_4000_3_0_1_nb','node')
#ws = nx.watts_strogatz_graph(4000, 20, 0.1)
#createGraphSAGEDataset(ws,'ws_4000_20_0_1_nb','node')
#ba = nx.barabasi_albert_graph(4000, 5)
#createGraphSAGEDataset(ba,'ba_4000_5_nb','node')


In [ ]:
# repeat with edge betweenness
er = nx.erdos_renyi_graph(100, 0.15)
createGraphSAGEDataset(er,'er_100_0_15_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(100, 0.45)
createGraphSAGEDataset(er,'er_100_0_45_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(1000, 0.15)
createGraphSAGEDataset(er,'er_1000_0_15_eb','edge', undirected=False)
er = nx.erdos_renyi_graph(1000, 0.45)
createGraphSAGEDataset(er,'er_1000_0_45_eb','edge', undirected=False)


ws = nx.watts_strogatz_graph(30, 3, 0.1)
createGraphSAGEDataset(ws,'ws_30_3_0_1_eb','edge', undirected=False)
ws = nx.watts_strogatz_graph(100, 3, 0.1)
createGraphSAGEDataset(ws,'ws_100_3_0_1_eb','edge', undirected=False)
ws = nx.watts_strogatz_graph(1000, 3, 0.1)
createGraphSAGEDataset(ws,'ws_1000_3_0_1_eb','edge', undirected=False)


ws = nx.watts_strogatz_graph(1000, 10, 0.1)
createGraphSAGEDataset(ws,'ws_1000_10_0_1_eb','edge', undirected=False)


ba = nx.barabasi_albert_graph(100, 5)
createGraphSAGEDataset(ba,'ba_100_5_eb','edge', undirected=False)
ba = nx.barabasi_albert_graph(1000, 5)
createGraphSAGEDataset(ba,'ba_1000_5_eb','edge', undirected=False)


#er = nx.erdos_renyi_graph(4000, 0.15)
#createRandomGraphDataset(er,'er_4000_0_15_eb','edge')
#er = nx.erdos_renyi_graph(4000, 0.35)
#createRandomGraphDataset(er,'er_4000_0_35_eb','edge')
#ws = nx.watts_strogatz_graph(4000, 3, 0.1)
#createRandomGraphDataset(ws,'ws_4000_3_0_1_eb','edge')
#ws = nx.watts_strogatz_graph(4000, 20, 0.1)
#createRandomGraphDataset(ws,'ws_4000_20_0_1_eb','edge')
#ba = nx.barabasi_albert_graph(4000, 5)
#createRandomGraphDataset(ba,'ba_4000_5_eb','edge')

In [ ]:
# graphs for the inductive setting

for i in range(20):
    er = nx.erdos_renyi_graph(100, 0.15)
    createGraphSAGEDataset(er,'ind_er_100_0_15_eb_i_'+str(i),'edge', undirected=False)
    er = nx.erdos_renyi_graph(100, 0.45)
    createGraphSAGEDataset(er,'ind_er_100_0_45_eb_i_'+str(i),'edge', undirected=False)
    #er = nx.erdos_renyi_graph(1000, 0.35)
    #createGraphSAGEDataset(er,'ind_er_1000_0_35_eb_i_'+str(i),'edge', undirected=False)

    ws = nx.watts_strogatz_graph(30, 3, 0.1)
    createGraphSAGEDataset(ws,'ws_30_3_0_1_eb_i_'+str(i),'edge', undirected=False)
    ws = nx.watts_strogatz_graph(100, 3, 0.1)
    createGraphSAGEDataset(ws,'ws_100_3_0_1_eb_i_'+str(i),'edge', undirected=False)
    #ws = nx.watts_strogatz_graph(1000, 3, 0.1)
    #createGraphSAGEDataset(ws,'ws_1000_3_0_1_eb','edge', undirected=False)
    #ws = nx.watts_strogatz_graph(1000, 10, 0.1)
    #createGraphSAGEDataset(ws,'ws_1000_10_0_1_eb_i_'+str(i),'edge', undirected=False)

    ba = nx.barabasi_albert_graph(100, 5)
    createGraphSAGEDataset(ba,'ba_100_5_eb_i_'+str(i),'edge', undirected=False)
    #ba = nx.barabasi_albert_graph(1000, 5)
    #createGraphSAGEDataset(ba,'ba_1000_5_eb','edge', undirected=False)


## 3. Benchmark datasets

In [ ]:

# plan
# 1. get graph from PyTorch Geom
# 2. transform to nx (pyTorchGeometricDatasetToNx)
# 3. call createGraphSAGEDataset(ba,'ba_1000_5','edge') 
#      or createGraphSAGEDataset(ba,'ba_1000_5','node')


def processDatasetsSingle(dname, dataset):
    G = dataset.data
    print(G)
    g = pyTorchGeometricDatasetToNx(G,i)
    i+=1
    print(i)
    createGraphSAGEDataset(g,'./precomputed-graphSAGE-TF/'+dname+'_'+str(i)+'_eb','edge', undirected=False) 
    createGraphSAGEDataset(g,'./precomputed-graphSAGE-TF/'+dname+'_'+str(i)+'_nd','node') 

        
def processDatasets(dname, dataset):
    # set size of batch to total size of graph here
    loader = DataLoader(dataset, shuffle=False)
    i = 0
    for G in loader:
        print(G)
        g = pyTorchGeometricDatasetToNx(G,i)
        i+=1
        print(i)
        createGraphSAGEDataset(g,'./precomputed-graphSAGE-TF/'+dname+'_'+str(i)+'_eb','edge', undirected=False) 
        createGraphSAGEDataset(g,'./precomputed-graphSAGE-TF/'+dname+'_'+str(i)+'_nd','node') 
    

#KarateClub
print("\nKarateClub")
dname='KarateClub'
dataset = loadDataset(dname)
processDatasets(dname, dataset)

#ENZYMES FROM TUDataset
print("\nTUDataset EnZYMES")
dname='TUDataset'
name='ENZYMES'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#PROTEINS FROM TUDataset
print("\nTUDataset PROTEINS")
dname='TUDataset'
name='PROTEINS'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#QM7b
print("\QM7B QM7B")
dataset = loadDataset('QM7b')
processDatasets(dname,dataset)

# Planetoid Cora
print("\n Planetoid Cora")
dname='Planetoid'
name='Cora'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name,dataset)



#---------------------------------------------------------------------


#MUTAG
#print("\n MUTAG")
#dname='Entities'
#name='MUTAG'
#dataset = loadDataset(dname,name)
#processDatasets(dname+'_'+name,dataset)



#PPI
#print("\PPI PPI")
#dname='PPI'
#dataset = loadDataset(dname)
#processDatasets(dname,dataset)



        
        


In [ ]:

#PROTEINS FROM TUDataset
print("\nTUDataset PROTEINS")
dname='TUDataset'
name='PROTEINS'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name, dataset)

#QM7b
print("\QM7B QM7B")
dataset = loadDataset('QM7b')
processDatasets(dname,dataset)

# Planetoid Cora
print("\n Planetoid Cora")
dname='Planetoid'
name='Cora'
dataset = loadDataset(dname,name)
processDatasets(dname+'_'+name,dataset)



